In [1]:
import pandas as pd
from AssessSurvey import LoadSurvey
import survey_utils
import survey_tables
import contingency_tables

survey_name = 'Capitol'

def process_category_for_regression(ques, names_str, names_str2, rm):
    df_first_media = pd.get_dummies(df[ques])
    df_first_media = df_first_media[[i for i in df_first_media.columns if i not in rm]]
    cols_dict = dict(zip(names_str,names_str2))
    df_first_media = df_first_media.rename(columns=dict(zip(names_str,names_str2)))
    return df_first_media

def create_second_order_capitol_riot_variables(df):
    cols = ['capitol_weak_strong',
    'capitol_weak_noeffect', 
    'capitol_strong_noeffect',
    'capitol_ordinal',
     'capitol_effect_noeffect']

    df1 = pd.DataFrame(columns = cols, index=df.index.values)

    c = cols[0]
    dd = df1[c]
    dd[df['capitol_weaker'] == 1] = 0
    dd[df['capitol_stronger'] == 1] = 1
    df1[c] = dd

    c = cols[1]
    dd = df1[c]
    dd[df['capitol_weaker'] == 1] = 0
    dd[df['capitol_noeffect'] == 1] = 1
    #dd[df['capitol_dontknow'] == 1] = 1
    df1[c] = dd

    c = cols[2]
    dd = df1[c]
    dd[df['capitol_noeffect'] == 1] = 0 
    #dd[df['capitol_dontknow'] == 1] = 0 
    dd[df['capitol_stronger'] == 1] = 1
    df1[c] = dd
    
    #c = cols[2]
    #dd = df1[c]
    #dd[df['capitol_noeffect'] == 1] = 0
    ##dd[df['capitol_stronger'] == 1] = 1
    #df1[c] = dd
    
    c = cols[3]
    dd = df1[c]
    dd[df['capitol_weaker'] == 1] = 0
    dd[df['capitol_noeffect'] == 1] = 0.5
    #dd[df['capitol_dontknow'] == 1] = 0.5
    dd[df['capitol_stronger'] == 1] = 1
    df1[c] = dd
    
    c = cols[4]
    dd = df1[c]
    dd[df['capitol_weaker'] == 1] = 1
    dd[df['capitol_noeffect'] == 1] = 0
    #dd[df['capitol_dontknow'] == 1] = 0 
    dd[df['capitol_stronger'] == 1] = 1
    df1[c] = dd

    df = pd.concat([df1,df],1)

    return df

def create_second_order_country_future_variables(df):

    # country_future_dontknow
    # country_future_russia
    # country_future_eu

    cols = ['country_future_russia_eu',
    'country_future_dontknow_eu', 
    'country_future_russia_dontknow',
    'country_future_ordinal'
    ]

    df1 = pd.DataFrame(columns = cols, index=df.index.values)

    c = cols[0]
    dd = df1[c]
    dd[df['country_future_russia'] == 1] = 0
    dd[df['country_future_eu'] == 1] = 1
    df1[c] = dd

    c = cols[1]
    dd = df1[c]
    dd[df['country_future_dontknow'] == 1] = 0
    dd[df['country_future_eu'] == 1] = 1
    df1[c] = dd

    c = cols[2]
    dd = df1[c]
    dd[df['country_future_russia'] == 1] = 0
    dd[df['country_future_dontknow'] == 1] = 1
    df1[c] = dd

    c = cols[3]
    dd = df1[c]
    dd[df['country_future_russia'] == 1] = -1
    dd[df['country_future_dontknow'] == 1] = 0
    dd[df['country_future_eu'] == 1] = 1
    df1[c] = dd

    df = pd.concat([df1,df],1)

    return df


def create_second_order_media_variables(df):
    
    for tt in ['source','trust']:

        df[tt+'_russian'] = (df[tt+"_russian_media"] + df[tt+"_russian_digital_media"]
                                   - df[tt+"_national_digital_media"] - df[tt+"_national_mass_media"] )

        df[tt+'_digital'] = (df[tt+"_national_digital_media"] + df[tt+"_russian_digital_media"]  + df[tt+"_social_media"]
                                 - df[tt+"_national_mass_media"] - df[tt+"_russian_media"])

    return df

In [2]:
#country_vec = [, 'Ukraine','Belarus']

country = 'Belarus'

survey = LoadSurvey(survey_name, country)
df = survey.data

print(df.columns)
rm = ['Hard to say/ Don’t know','nan']
ques = 'q4'

names_str = ['Social media: Instagram',
 'Online mass media',
 'Traditional Media (i. e. , newspapers, television, radio)',
 'Social media: Facebook',
 'Social media: YouTube',
 'Social media: Twitter',
 'Friends or family members',
 'Social media: Vkontakte',
 'Social media Telegram',
 'Social media: Odnoklassniki']

names_str2 = ['firstmedia_social_media_instagram', 'firstmedia_online_mass_media', 
              'firstmedia_traditional_media', 'firstmedia_social_media_facebook', 
              'firstmedia_social_media_youtube', 'firstmedia_social_media_twitter',
              'firstmedia_friends_and_family', 'firstmedia_social_media_vkontakte',
              'firstmedia_social_media_telegram', 'firstmedia_social_media_odnoklassniki']

df_first_media = process_category_for_regression(ques, names_str, names_str2, rm)


DD = dict(zip(['Not mentioned', 'Mentioned'],[0, 1]))

cols_ii = ['q1_'+ str(i) for i in range(1,7)] + ['q2_'+ str(i) for i in range(1,7)] 

names = []
for d in ['source','trust']:
    names.extend([d+i for i in ['_national_mass_media', '_social_media', '_russian_media', '_national_digital_media', 
    '_russian_digital_media', '_people_you_know'] ])
    
df_media = df[cols_ii]
df_media = df_media.rename(columns=dict(zip(cols_ii, names)))
df_media = df_media.replace(DD)


rm = []
ques = 'q5'

names_str = ['These events won’t affect the role of the U. S.',
 'Don’t know',
 'The role of the U. S. will become weaker',
 'The role of the U. S. will become stronger']

names_str2 = ['capitol_noeffect','capitol_dontknow','capitol_weaker','capitol_stronger' ]
df_capitol = process_category_for_regression(ques, names_str, names_str2, rm)

rm = ['Refusal to answer']
ques = 'q10'

names_str = ['Joining the European Union',
 'Hard to say/DK',
 'Improving relations with Russia',
 ]

names_str2 = ['country_future_eu', 'country_future_dontknow',
                  'country_future_russia']

df_country_future = process_category_for_regression(ques, names_str, names_str2, rm)

age = df['d1']

education = df['d3']

#belarus
edu_ordered_int = [6,5,4,3,2,1, None, None]
edu_ordered_cat = ['Higher university',
 'Incomplete higher (at least 3 years)',
 'Advanced secondary/vocational (technikum, college, professional)',
 'General secondary y (secondary school, lyceum, gymnasium)',
 'Incomplete secondary (basic level)',
 'Elementary or less',
 'Refused',
 'Don’t know']

education = education.replace(dict(zip(edu_ordered_cat, edu_ordered_int)))

rural = df['d5'].astype(str)
rural[df['d5'].values == 'Rural'] = 1
rural[df['d5'].values != 'Rural'] = 0

#belarus
wealth = df['d8']

wealth_ordered_int = [6,5,4,3,2,1, None, None]
wealth_ordered_cat = ['We can afford almost anything',
 'We can buy a car, but we cannot say that we can afford anything',
 'We can buy some things like TV or fridge, but not a car',
 'We have enough money for food and clothing, but buying more expensive goods – like TV or fridge causes problems for',
 'We have enough money for food, but not for clothing',
 'We do not have enough money even for food',
 'Refused',
 'Hard to say/Don’t know']

wealth = wealth.replace(dict(zip(wealth_ordered_cat, wealth_ordered_int)))

female = df['d2'].astype(str)

female[df['d2'] == 'Female'] = 1
female[df['d2'] != 'Female'] = 0


df_dem = pd.concat([age, education, rural, wealth, female],1)
df_dem.columns = ['age','education', 'rural', 'wealth', 'female']

df = pd.concat( [df_capitol, df_first_media, df_media, df_country_future, df_dem] , 1)

df = create_second_order_country_future_variables(df)
df = create_second_order_capitol_riot_variables(df)
df = create_second_order_media_variables(df)

fn = '{c}_regression.csv'.format(c=country)
df.to_csv(fn)

load data from Belarus...
number of participants:  1014
number of entries:  40
Index(['id', 'q1_1', 'q1_2', 'q1_3', 'q1_4', 'q1_5', 'q1_6', 'q1_7', 'q1_98',
       'q1_99', 'q2_1', 'q2_2', 'q2_3', 'q2_4', 'q2_5', 'q2_6', 'q2_7',
       'q2_98', 'q2_99', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'd1',
       'd2', 'd3', 'd4', 'd5', 'dd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'Weight',
       'age'],
      dtype='object')


In [3]:
#country_vec = [, 'Ukraine','Belarus']
country = 'Georgia'

survey = LoadSurvey(survey_name, country)
df = survey.data
    
rm = ['Don’t know']
ques = 'q4'

names_str = ['\xa0  Online mass media',
 '\xa0 Friends or family members',
 '\xa0 Social media: Instagram',
 '\xa0\xa0"Traditional Media [i.e., newspapers, television, radio]"',
 '\xa0\xa0Social media: Facebook',
 '\xa0\xa0Social media: Telegram',
 '\xa0\xa0Social media: Twitter',
 '\xa0\xa0Social media: YouTube']

names_str2 = ['firstmedia_online_mass_media',
                  'firstmedia_friends_and_family', 'firstmedia_social_media_instagram',
                  'firstmedia_traditional_media',
                  'firstmedia_social_media_facebook', 'firstmedia_social_media_telegram',
                  'firstmedia_social_media_twitter', 'firstmedia_social_media_youtube']

df_first_media = process_category_for_regression(ques, names_str, names_str2, rm)

DD = dict(zip(['Not mentioned', 'Mentioned'],[0, 1]))

cols_ii = ['q1_'+ str(i) for i in range(1,7)] + ['q2_'+ str(i) for i in range(1,7)] 

names = []
for d in ['source','trust']:
    names.extend([d+i for i in ['_national_mass_media', '_social_media', '_russian_media', '_national_digital_media', 
    '_russian_digital_media', '_people_you_know'] ])
    
df_media = df[cols_ii]
df_media = df_media.rename(columns=dict(zip(cols_ii, names)))
df_media = df_media.replace(DD)

rm = []
ques = 'q5'

names_str = ['The role of the U.S.  will become weaker',
 '\xa0"Don’t know"',
 '\xa0"The role of the U.S. will become stronger "',
 '\xa0"These events won’t affect the role of the U.S."']

names_str2 = ['capitol_weaker',
                  'capitol_dontknow', 'capitol_stronger',
                  'capitol_noeffect']

df_capitol = process_category_for_regression(ques, names_str, names_str2, rm)


rm = ['Refuse to answer']
ques = 'q10'

names_str = ['Hard to say/DK',
 '\xa0Improving relations with Russia',
 '\xa0Joining the European Union']

names_str2 = ['country_future_dontknow',
                  'country_future_russia', 'country_future_eu']

df_country_future = process_category_for_regression(ques, names_str, names_str2, rm)

age = df['d1']

education = df['d3']

#Georgia
edu_ordered_int = [6,5,4,3,2,1]
edu_ordered_cat = ['Higher university',
 'Incomplete higher [at least 3 years]',
 'Advanced secondary/vocational [technikum, college, professional]',
 'General secondary y [secondary school, lyceum, gymnasium]',
 'Incomplete secondary [basic level]',
 'Elementary or less']


education = education.replace(dict(zip(edu_ordered_cat, edu_ordered_int)))

rural = df['d5'].astype(str)
rural[df['d5'].values == 'Rural'] = 1
rural[df['d5'].values != 'Rural'] = 0

#belarus
wealth = df['d8']

#Georgia
wealth_ordered_int = [6,5,4,3,2,1, None, None]
wealth_ordered_cat = ['We can afford almost anything',
 'We can buy a car, but we cannot say that we can afford anything',
 'We can buy some things like TV or fridge, but not a car',
 'We have enough money for food and clothing, but buying more expensive goods –like TV or fridge causes problems for us',
 'We have enough money for food, but not for clothing',
 'We do not have enough money even for food',
 'Refused',
 'Don’t know']

wealth = wealth.replace(dict(zip(wealth_ordered_cat, wealth_ordered_int)))

female = df['d2'].astype(str)

female[df['d2'] == 'Female'] = 1
female[df['d2'] != 'Female'] = 0


df_dem = pd.concat([age, education, rural, wealth, female],1)
df_dem.columns = ['age','education', 'rural', 'wealth', 'female']

df = pd.concat( [df_capitol, df_first_media, df_media, df_country_future, df_dem] , 1)

df = create_second_order_country_future_variables(df)
df = create_second_order_capitol_riot_variables(df)
df = create_second_order_media_variables(df)

fn = '{c}_regression.csv'.format(c=country)
df.to_csv(fn)

load data from Georgia...
number of participants:  1000
number of entries:  42


In [4]:
country = 'Ukraine'

survey = LoadSurvey(survey_name, country)
df = survey.data

print(df.columns)
rm = ['Hard to say/Don’t know','nan']
ques = 'q4'

names_str = ['Instagram',
 'Online mass media',
 'Traditional Media (i.e., newspapers, television, radio)',
 'Facebook',
 'YouTube',
 'Twitter',
 'Friends or family members',
 'Vkontakte',
 'Telegram',
 'Odnoklassniki']

names_str2 = ['firstmedia_social_media_instagram', 'firstmedia_online_mass_media', 
              'firstmedia_traditional_media', 'firstmedia_social_media_facebook', 
              'firstmedia_social_media_youtube', 'firstmedia_social_media_twitter',
              'firstmedia_friends_and_family', 'firstmedia_social_media_vkontakte',
              'firstmedia_social_media_telegram', 'firstmedia_social_media_odnoklassniki']

df_first_media = process_category_for_regression(ques, names_str, names_str2, rm)

DD = dict(zip(['Not Mentioned', 'Mentioned'],[0, 1]))

cols_ii = ['q1_'+ str(i) for i in range(1,7)] + ['q2_'+ str(i) for i in range(1,7)] 

names = []
for d in ['source','trust']:
    names.extend([d+i for i in ['_national_mass_media', '_social_media', '_russian_media', '_national_digital_media', 
    '_russian_digital_media', '_people_you_know'] ])
    
df_media = df[cols_ii]
df_media = df_media.rename(columns=dict(zip(cols_ii, names)))
df_media = df_media.replace(DD)

rm = []
ques = 'q5'

names_str = ['These events won’t affect the role of the U.S.',
 'Don’t know',
 'The role of the U.S.  will become weaker',
 'The role of the U.S. will become stronger']

names_str2 = ['capitol_noeffect','capitol_dontknow','capitol_weaker','capitol_stronger' ]
df_capitol = process_category_for_regression(ques, names_str, names_str2, rm)

rm = ['Refusal to answer']
ques = 'q10'

names_str = ['Joining the European Union',
 'Hard to say/DK',
 'Improving relations with Russia',
 ]

names_str2 = ['country_future_eu', 'country_future_dontknow',
                  'country_future_russia']

df_country_future = process_category_for_regression(ques, names_str, names_str2, rm)

age = df['d1']

education = df['d3']

#Ukraine
edu_ordered_int = [6,5,4,3,2,1, None]
edu_ordered_cat = ['Higher university',
 'Incomplete higher (at least 3 years)',
'Advanced secondary/vocational (technikum, college, professional)',
'General secondary y (secondary school, lyceum, gymnasium)',
'Incomplete secondary (basic level)',
'Elementary or less',
'Refused']

education = education.replace(dict(zip(edu_ordered_cat, edu_ordered_int)))

rural = df['d5'].astype(str)
rural[df['d5'].values == 'Rural'] = 1
rural[df['d5'].values != 'Rural'] = 0

wealth = df['d8']

#Ukraine
wealth_ordered_int = [6,5,4,3,2,1, None, None]
wealth_ordered_cat = ['We can afford almost anything',
 'We can buy a car, but we cannot say that we can afford anything',
 'We can buy some things like TV or fridge, but not a car',
 'We have enough money for food and clothing, but buying more expensive goods –  like TV or fridge causes problems for us',
 'We have enough money for food, but not for clothing',
 'We do not have enough money even for food',
 'Refused',
 'Don’t know']

wealth = wealth.replace(dict(zip(wealth_ordered_cat, wealth_ordered_int)))

female = df['d2'].astype(str)

female[df['d2'] == 'Female'] = 1
female[df['d2'] != 'Female'] = 0


df_dem = pd.concat([age, education, rural, wealth, female],1)
df_dem.columns = ['age','education', 'rural', 'wealth', 'female']

df = pd.concat( [df_capitol, df_first_media, df_media, df_country_future, df_dem] , 1)

df = create_second_order_country_future_variables(df)
df = create_second_order_capitol_riot_variables(df)
df = create_second_order_media_variables(df)

fn = '{c}_regression.csv'.format(c=country)
df.to_csv(fn)

load data from Ukraine...
number of participants:  2000
number of entries:  41
Index(['n_ank', 'p3', 'q1_1', 'q1_2', 'q1_3', 'q1_4', 'q1_5', 'q1_6', 'q1_7',
       'q1_98', 'q1_99', 'q2_1', 'q2_2', 'q2_3', 'q2_4', 'q2_5', 'q2_6',
       'q2_7', 'q2_98', 'q2_99', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9',
       'q10', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10',
       'age', 'gyt', 'Weight'],
      dtype='object')


In [1]:
'''
create a description of the Python code in this notebook
'''



NameError: name 'd' is not defined

In [ ]:
#country_vec = [, 'Ukraine','Belarus']

age = df['d1']

education = df['d3']

#belarus
edu_ordered_int = [6,5,4,3,2,1, None, None]
edu_ordered_cat = ['Higher university',
 'Incomplete higher (at least 3 years)',
 'Advanced secondary/vocational (technikum, college, professional)',
 'General secondary y (secondary school, lyceum, gymnasium)',
 'Incomplete secondary (basic level)',
 'Elementary or less',
 'Refused',
 'Don’t know']

education = education.replace(dict(zip(edu_ordered_cat, edu_ordered_int)))

rural = df['d5'].astype(str)
rural[df['d5'].values == 'Rural'] = 1
rural[df['d5'].values != 'Rural'] = 0

#belarus
wealth = df['d8']

wealth_ordered_int = [6,5,4,3,2,1, None, None]
wealth_ordered_cat = ['We can afford almost anything',
 'We can buy a car, but we cannot say that we can afford anything',
 'We can buy some things like TV or fridge, but not a car',
 'We have enough money for food and clothing, but buying more expensive goods – like TV or fridge causes problems for',
 'We have enough money for food, but not for clothing',
 'We do not have enough money even for food',
 'Refused',
 'Hard to say/Don’t know']

wealth = wealth.replace(dict(zip(wealth_ordered_cat, wealth_ordered_int)))

female = df['d2'].astype(str)

female[df['d2'] == 'Female'] = 1
female[df['d2'] != 'Female'] = 0


df_dem = pd.concat([age, education, rural, wealth, female],1)
df_dem.columns = ['age','education', 'rural', 'wealth', 'female']

df = pd.concat( [df_capitol, df_first_media, df_media, df_country_future, df_dem] , 1)

df = create_second_order_country_future_variables(df)
df = create_second_order_capitol_riot_variables(df)

fn = '{c}_regression.csv'.format(c=country)
df.to_csv(fn)

In [87]:


'Age'
'Education Level'
'Urban'
'Wealthier'
'Female'








load data from Belarus...
number of participants:  1014
number of entries:  40


0       1
1       1
2       0
3       1
4       0
       ..
1009    1
1010    1
1011    0
1012    0
1013    1
Name: d2, Length: 1014, dtype: object

Index(['capitol_weak_strong', 'capitol_weak_noeffect',
       'capitol_strong_noeffect', 'capitol_ordinal',
       'country_future_russia_eu', 'country_future_dontknow_eu',
       'country_future_russia_dontknow', 'capitol_weaker', 'capitol_dontknow',
       'capitol_stronger', 'capitol_noeffect', 'firstmedia_online_mass_media',
       'firstmedia_friends_and_family', 'firstmedia_social_media_instagram',
       'firstmedia_traditional_media', 'firstmedia_social_media_facebook',
       'firstmedia_social_media_telegram', 'firstmedia_social_media_twitter',
       'firstmedia_social_media_youtube', 'source_national_mass_media',
       'source_social_media', 'source_russian_media',
       'source_national_digital_media', 'source_russian_digital_media',
       'source_people_you_know', 'trust_national_mass_media',
       'trust_social_media', 'trust_russian_media',
       'trust_national_digital_media', 'trust_russian_digital_media',
       'trust_people_you_know', 'country_future_dontknow'

,country_future_russia_eu,country_future_dontknow_eu,country_future_russia_dontknow,capitol_weak_strong,capitol_weak_noeffect,capitol_strong_noeffect,capitol_weaker,capitol_dontknow,capitol_stronger,capitol_noeffect,...,source_people_you_know,trust_national_mass_media,trust_social_media,trust_russian_media,trust_national_digital_media,trust_russian_digital_media,trust_people_you_know,country_future_dontknow,country_future_russia,country_future_eu
0,NaN,0,1,NaN,NaN,NaN,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
1,0,NaN,0,NaN,NaN,NaN,0,1,0,0,...,0,1,1,0,1,0,0,0,1,0
2,1,1,NaN,0,0,NaN,1,0,0,0,...,0,1,0,0,1,0,1,0,0,1
3,1,1,NaN,NaN,NaN,NaN,0,1,0,0,...,1,0,1,0,0,0,1,0,0,1
4,1,1,NaN,NaN,NaN,NaN,0,1,0,0,...,0,1,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,0,1,NaN,NaN,NaN,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
996,1,1,NaN,0,0,NaN,1,0,0,0,...,1,0,0,0,0,0,1,0,0,1
997,1,1,NaN,1,NaN,1,0,0,1,0,...,0,1,1,0,0,1,0,0,0,1
998,1,1,NaN,NaN,1,0,0,0,0,1,...,1,1,1,0,0,0,1,0,0,1


In [ ]:
print(df[[i for i in df.columns if i.startswith('trust_')]].columns)


media_cons = ['trust', 'source']


mc = media_cons[0]
# country_future_dontknow
# country_future_russia
# country_future_eu

cols = [ mc + i for i in ['_russian_v_domestic',
'_mass_v_digital'] ]


df1 = pd.DataFrame(columns = cols, index=df.index.values)

c = cols[0]

c_nat
df[[trust_national_mass_media]]